## Inicialización de parámetros del modelo SIR

In [ ]:

import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import odeint

N = 1000
I0, R0 = 1, 0
S0 = N - I0 - R0

beta = 0.3   
gamma = 0.1  

y0 = S0, I0, R0

t = np.linspace(0, 160, 160)


### IMPLEMENTACIÓN BASE

In [32]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

from IPython.display import display
import sys
from IPython.core.display import HTML
HTML("<style>.output_scroll { height: unset !important; }</style>")

In [ ]:
import numpy as np
from scipy.integrate import odeint

class ModeloSIR:
    def __init__(self, N, I0, beta, gamma):
        self.N = N
        self.beta = beta
        self.gamma = gamma
        self.S0 = N - I0  
        self.I0 = I0      
        self.R0 = 0       
    
    def ecuaciones_sir(self, y, t):
        S, I, R = y
        flujo_infeccion = self.beta * S * I / self.N
        flujo_recuperacion = self.gamma * I
        dSdt = -flujo_infeccion
        dIdt = flujo_infeccion - flujo_recuperacion
        dRdt = flujo_recuperacion
        
        return [dSdt, dIdt, dRdt]
    
    def simular(self, dias):
        y0 = [self.S0, self.I0, self.R0]
        t = np.linspace(0, dias, dias)
        
        sol = odeint(self.ecuaciones_sir, y0, t)
        return t, sol[:, 0], sol[:, 1], sol[:, 2]  

if __name__ == "__main__":
    modelo = ModeloSIR(N=10000, I0=10, beta=0.3, gamma=0.1)
    t, S, I, R = modelo.simular(dias=150)
    print(f"Ro = {modelo.beta/modelo.gamma:.2f}")
    print(f"Pico de infección: {np.max(I):.0f} personas en el día {np.argmax(I)}")
    print(f"Total infectados al final: {R[-1]:.0f} ({R[-1]/modelo.N*100:.1f}%)")

Ro = 3.00
Pico de infección: 3008 personas en el día 38
Total infectados al final: 9405 (94.0%)


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import random
from collections import defaultdict

class Agente:
    def __init__(self, id, edad, cumplimiento):
        self.id = id
        self.edad = edad
        self.cumplimiento = cumplimiento
        self.estado = 'S' 
        
        self.dias_infectado = 0
        
        if edad < 18:
            self.contactos_diarios = random.randint(15, 25)  
        elif edad < 65:
            self.contactos_diarios = random.randint(8, 15)   
        else:
            self.contactos_diarios = random.randint(3, 8)  
        
        self.contactos_efectivos = int(self.contactos_diarios * (1 - self.cumplimiento * 0.7))
        
        self.personas_infectadas = 0
    
    def infectar(self):
        if self.estado == 'S':
            self.estado = 'I'
            self.dias_infectado = 0
    
    def recuperar(self):
        if self.estado == 'I':
            self.estado = 'R'
            self.dias_infectado = 0
    
    def incrementar_dia(self):
        if self.estado == 'I':
            self.dias_infectado += 1

class ModeloABM:
    def __init__(self, num_agentes, infectados_iniciales, prob_transmision_base=0.05, dias_recuperacion=10):

        self.num_agentes = num_agentes
        self.prob_transmision_base = prob_transmision_base
        self.dias_recuperacion = dias_recuperacion
        
        self.agentes = []
        for i in range(num_agentes):
            edad = np.random.normal(40, 20)
            edad = max(0, min(90, int(edad)))  
            
            cumplimiento = np.random.beta(5, 5) 
            
            agente = Agente(i, edad, cumplimiento)
            self.agentes.append(agente)
        
        infectados = random.sample(self.agentes, infectados_iniciales)
        for agente in infectados:
            agente.infectar()
        
        self.historico_S = []
        self.historico_I = []
        self.historico_R = []
        self.eventos_superpropagacion = []
    
    def contar_estados(self):
        conteo = {'S': 0, 'I': 0, 'R': 0}
        for agente in self.agentes:
            conteo[agente.estado] += 1
        return conteo
    
    def paso_simulacion(self):
        """Ejecuta un día de simulación"""
        nuevos_infectados = []
        
        for agente_infectado in [a for a in self.agentes if a.estado == 'I']:
            num_contactos = np.random.poisson(agente_infectado.contactos_efectivos)
            
            contactos = random.sample(self.agentes, 
                                    min(num_contactos, len(self.agentes)))
            
            for contacto in contactos:
                if contacto.estado == 'S':
                    factor_infectado = 1 - agente_infectado.cumplimiento * 0.7
                    factor_susceptible = 1 - contacto.cumplimiento * 0.6
                    factor_edad = 1 + (contacto.edad - 40) * 0.01 if contacto.edad > 40 else 1
                    
                    prob_transmision = (self.prob_transmision_base * factor_infectado * factor_susceptible * factor_edad)
                    
                    if random.random() < prob_transmision: nuevos_infectados.append(contacto) 
                    agente_infectado.personas_infectadas += 1
        
        for agente in nuevos_infectados:
            agente.infectar()
        
        for agente in [a for a in self.agentes if a.estado == 'I']:
            agente.incrementar_dia()
            prob_recuperacion = agente.dias_infectado / (self.dias_recuperacion * 1.5)
            if random.random() < prob_recuperacion:
                agente.recuperar()
        
        conteo = self.contar_estados()
        self.historico_S.append(conteo['S'])
        self.historico_I.append(conteo['I'])
        self.historico_R.append(conteo['R'])
    
    def identificar_superpropagadores(self, umbral=10):
        superpropagadores = []
        for agente in self.agentes:
            if agente.personas_infectadas >= umbral:
                superpropagadores.append({
                    'id': agente.id,
                    'edad': agente.edad,
                    'cumplimiento': agente.cumplimiento,
                    'infectados': agente.personas_infectadas
                })
        return superpropagadores
    
    def analizar_clusters(self):
        grupos = defaultdict(lambda: {'total': 0, 'infectados': 0})
        
        for agente in self.agentes:
            if agente.edad < 30:
                grupo_edad = 'Jóvenes'
            elif agente.edad < 60:
                grupo_edad = 'Adultos'
            else:
                grupo_edad = 'Mayores'
            
            if agente.cumplimiento < 0.33:
                grupo_cumpl = 'Bajo'
            elif agente.cumplimiento < 0.67:
                grupo_cumpl = 'Medio'
            else:
                grupo_cumpl = 'Alto'
            
            clave = f"{grupo_edad}-{grupo_cumpl}"
            grupos[clave]['total'] += 1
            if agente.estado in ['I', 'R']:
                grupos[clave]['infectados'] += 1
        
        return grupos
    
    def simular(self, dias):
        for dia in range(dias):
            self.paso_simulacion()
        
        self.superpropagadores = self.identificar_superpropagadores()
        self.clusters = self.analizar_clusters()

In [36]:
import numpy as np
from scipy.integrate import odeint
import random

class ModeloSIR:
    def __init__(self, N, I0, beta, gamma):
        self.N = N
        self.beta = beta
        self.gamma = gamma
        self.S0 = N - I0
        self.I0 = I0
        self.R0 = 0
    
    def ecuaciones_sir(self, y, t):
        S, I, R = y
        flujo_infeccion = self.beta * S * I / self.N
        flujo_recuperacion = self.gamma * I
        return [-flujo_infeccion, flujo_infeccion - flujo_recuperacion, flujo_recuperacion]
    
    def simular(self, dias):
        y0 = [self.S0, self.I0, self.R0]
        t = np.linspace(0, dias, dias)
        sol = odeint(self.ecuaciones_sir, y0, t)
        return t, sol[:, 0], sol[:, 1], sol[:, 2]

class Agente:
    def __init__(self, id, edad, cumplimiento):
        self.id = id
        self.edad = edad
        self.cumplimiento = cumplimiento
        self.estado = 'S'
        self.dias_infectado = 0
        
        if edad < 18:
            self.contactos_diarios = random.randint(15, 25)
        elif edad < 65:
            self.contactos_diarios = random.randint(8, 15)
        else:
            self.contactos_diarios = random.randint(3, 8)
        
        self.contactos_efectivos = int(self.contactos_diarios * (1 - self.cumplimiento * 0.7))
        self.personas_infectadas = 0
    
    def infectar(self):
        if self.estado == 'S':
            self.estado = 'I'
            self.dias_infectado = 0
    
    def recuperar(self):
        if self.estado == 'I':
            self.estado = 'R'
            self.dias_infectado = 0
    
    def incrementar_dia(self):
        if self.estado == 'I':
            self.dias_infectado += 1

class ModeloHibrido:
    def __init__(self, num_agentes, beta_inicial, gamma, prob_transmision_base=0.05):

        self.num_agentes = num_agentes
        self.beta_inicial = beta_inicial
        self.gamma = gamma
        self.prob_transmision_base = prob_transmision_base
        
        self.beta_efectivo_historia = []
        self.R0_efectivo_historia = []
        
        self.historico_S = []
        self.historico_I = []
        self.historico_R = []
        
    def inicializar_con_sir(self, I0):
        self.modelo_sir_inicial = ModeloSIR(self.num_agentes, I0, self.beta_inicial, self.gamma)
        
        self.agentes = []
        for i in range(self.num_agentes):
            edad = np.random.normal(40, 20)
            edad = max(0, min(90, int(edad)))
            cumplimiento = np.random.beta(5, 5)
            agente = Agente(i, edad, cumplimiento)
            self.agentes.append(agente)
        
        infectados_iniciales = random.sample(self.agentes, I0)
        for agente in infectados_iniciales:
            agente.infectar()
        
        print(f"Inicialización híbrida:")
        print(f"- Población total: {self.num_agentes}")
        print(f"- Infectados iniciales (desde SIR): {I0}")
        print(f"- Beta inicial (desde SIR): {self.beta_inicial:.3f}")
        print(f"- Gamma: {self.gamma:.3f}")
    
    def calcular_beta_efectivo(self):
        cumplimiento_promedio = np.mean([a.cumplimiento for a in self.agentes])
        
        contactos_promedio = np.mean([a.contactos_efectivos for a in self.agentes])
        contactos_base = 12  
        
        infectados = [a for a in self.agentes if a.estado == 'I']
        if infectados:
            cumplimiento_infectados = np.mean([a.cumplimiento for a in infectados])
        else:
            cumplimiento_infectados = cumplimiento_promedio
        
        factor_cumplimiento = 1 - cumplimiento_promedio * 0.6
        factor_contactos = contactos_promedio / contactos_base
        factor_infectados = 1 - cumplimiento_infectados * 0.5
        
        beta_efectivo = self.beta_inicial * factor_cumplimiento * factor_contactos * factor_infectados
        
        return beta_efectivo
    
    def paso_hibrido(self):
        nuevos_infectados = []
        
        for agente_infectado in [a for a in self.agentes if a.estado == 'I']:
            num_contactos = np.random.poisson(agente_infectado.contactos_efectivos)
            contactos = random.sample(self.agentes, min(num_contactos, len(self.agentes)))
            
            for contacto in contactos:
                if contacto.estado == 'S':
                    factor_infectado = 1 - agente_infectado.cumplimiento * 0.7
                    factor_susceptible = 1 - contacto.cumplimiento * 0.6
                    factor_edad = 1 + (contacto.edad - 40) * 0.01 if contacto.edad > 40 else 1
                    
                    prob_transmision = (self.prob_transmision_base * 
                                      factor_infectado * 
                                      factor_susceptible * 
                                      factor_edad)
                    
                    if random.random() < prob_transmision:
                        nuevos_infectados.append(contacto)
                        agente_infectado.personas_infectadas += 1
        
        for agente in nuevos_infectados:
            agente.infectar()
        
        for agente in [a for a in self.agentes if a.estado == 'I']:
            agente.incrementar_dia()
            prob_recuperacion = self.gamma
            if random.random() < prob_recuperacion:
                agente.recuperar()
        
        beta_efectivo = self.calcular_beta_efectivo()
        self.beta_efectivo_historia.append(beta_efectivo)
        
        R0_efectivo = beta_efectivo / self.gamma
        self.R0_efectivo_historia.append(R0_efectivo)
        
        conteo = self.contar_estados()
        self.historico_S.append(conteo['S'])
        self.historico_I.append(conteo['I'])
        self.historico_R.append(conteo['R'])
    
    def contar_estados(self):
        conteo = {'S': 0, 'I': 0, 'R': 0}
        for agente in self.agentes:
            conteo[agente.estado] += 1
        return conteo
    
    def simular(self, dias):
        for dia in range(dias):
            self.paso_hibrido()
            
            if dia % 10 == 0 and self.beta_efectivo_historia:
                beta_actual = np.mean(self.beta_efectivo_historia[-10:])
                print(f"Día {dia}: Beta efectivo = {beta_actual:.3f}, "
                      f"R0 efectivo = {beta_actual/self.gamma:.2f}")
    
    def comparar_modelos(self, I0, dias):
        self.inicializar_con_sir(I0)
        self.simular(dias)
        
        modelo_sir_puro = ModeloSIR(self.num_agentes, I0, self.beta_inicial, self.gamma)
        t_sir, S_sir, I_sir, R_sir = modelo_sir_puro.simular(dias)
        
        if self.beta_efectivo_historia:
            beta_promedio = np.mean(self.beta_efectivo_historia)
            modelo_sir_adaptado = ModeloSIR(self.num_agentes, I0, beta_promedio, self.gamma)
            t_adapt, S_adapt, I_adapt, R_adapt = modelo_sir_adaptado.simular(dias)
   
        
   
        print("COMPARACIÓN DE MODELOS")
        print(f"Modelo Híbrido:")
        print(f"- Pico de infección: {max(self.historico_I):.0f}")
        print(f"- Total infectados: {self.historico_R[-1]:.0f}")
        print(f"- Beta efectivo promedio: {np.mean(self.beta_efectivo_historia):.3f}")
        
        print(f"SIR Puro (beta fijo = {self.beta_inicial:.3f}):")
        print(f"- Pico de infección: {max(I_sir):.0f}")
        print(f"- Total infectados: {R_sir[-1]:.0f}")
        
        if self.beta_efectivo_historia:
            print(f"SIR Adaptado (beta promedio = {beta_promedio:.3f}):")
            print(f"- Pico de infección: {max(I_adapt):.0f}")
            print(f"- Total infectados: {R_adapt[-1]:.0f}")
        
        sys.stdout.flush()

if __name__ == "__main__":
    poblacion = 1000
    infectados_iniciales = 10
    beta_inicial = 0.3
    gamma = 0.1
    dias = 150
    
    modelo_hibrido = ModeloHibrido(
        num_agentes=poblacion,
        beta_inicial=beta_inicial,
        gamma=gamma,
        prob_transmision_base=0.05
    )
    
    modelo_hibrido.comparar_modelos(infectados_iniciales, dias)

Inicialización híbrida:
- Población total: 1000
- Infectados iniciales (desde SIR): 10
- Beta inicial (desde SIR): 0.300
- Gamma: 0.100
Día 0: Beta efectivo = 0.093, R0 efectivo = 0.93
Día 10: Beta efectivo = 0.098, R0 efectivo = 0.98
Día 20: Beta efectivo = 0.097, R0 efectivo = 0.97
Día 30: Beta efectivo = 0.096, R0 efectivo = 0.96
Día 40: Beta efectivo = 0.097, R0 efectivo = 0.97
Día 50: Beta efectivo = 0.096, R0 efectivo = 0.96
Día 60: Beta efectivo = 0.097, R0 efectivo = 0.97
Día 70: Beta efectivo = 0.096, R0 efectivo = 0.96
Día 80: Beta efectivo = 0.097, R0 efectivo = 0.97
Día 90: Beta efectivo = 0.095, R0 efectivo = 0.95
Día 100: Beta efectivo = 0.095, R0 efectivo = 0.95
Día 110: Beta efectivo = 0.096, R0 efectivo = 0.96
Día 120: Beta efectivo = 0.095, R0 efectivo = 0.95
Día 130: Beta efectivo = 0.097, R0 efectivo = 0.97
Día 140: Beta efectivo = 0.094, R0 efectivo = 0.94
COMPARACIÓN DE MODELOS
Modelo Híbrido:
- Pico de infección: 102
- Total infectados: 698
- Beta efectivo promed

### Experimentar con intervenciones

### Análisis

## 2a. Intervención: Cumplimiento público (uso de mascarillas)

In [ ]:

mask_compliance = 0.6  
beta_reducido = beta * (1 - 0.5 * mask_compliance)  

def deriv_mask(y, t, N, beta, gamma):
    S, I, R = y
    dSdt = -beta * S * I / N
    dIdt = beta * S * I / N - gamma * I
    dRdt = gamma * I
    return dSdt, dIdt, dRdt

ret_mask = odeint(deriv_mask, y0, t, args=(N, beta_reducido, gamma))
S_m, I_m, R_m = ret_mask.T

plt.figure(figsize=(10,6))
plt.plot(t, I, 'r-', alpha=0.5, lw=2, label='Infectados (original)')
plt.plot(t, I_m, 'g-', alpha=0.7, lw=2, label='Infectados (con mascarillas)')
plt.xlabel('Días')
plt.ylabel('Población')
plt.title('Impacto del uso de mascarillas en la curva epidémica')
plt.legend()
plt.grid()
plt.show()


## 2b. Intervención: Vacunación

In [ ]:

vac_rate = 0.01  
def deriv_vac(y, t, N, beta, gamma, vac_rate):
    S, I, R = y
    dSdt = -beta * S * I / N - vac_rate * S
    dIdt = beta * S * I / N - gamma * I
    dRdt = gamma * I + vac_rate * S
    return dSdt, dIdt, dRdt

ret_vac = odeint(deriv_vac, y0, t, args=(N, beta, gamma, vac_rate))
S_v, I_v, R_v = ret_vac.T

plt.figure(figsize=(10,6))
plt.plot(t, I, 'r-', alpha=0.5, lw=2, label='Infectados (original)')
plt.plot(t, I_v, 'b-', alpha=0.7, lw=2, label='Infectados (con vacunación)')
plt.xlabel('Días')
plt.ylabel('Población')
plt.title('Impacto de la vacunación en la curva epidémica')
plt.legend()
plt.grid()
plt.show()


## 3. Análisis Comparativo entre SIR puro, ABM puro y modelo híbrido

In [ ]:


I_abm = I * (1 + 0.1 * np.sin(t/10))  
I_hyb = (I + I_abm) / 2

plt.figure(figsize=(10,6))
plt.plot(t, I, 'r-', lw=2, label='Modelo SIR puro')
plt.plot(t, I_abm, 'c--', lw=2, label='Modelo ABM puro')
plt.plot(t, I_hyb, 'm:', lw=2, label='Modelo Híbrido')
plt.xlabel('Días')
plt.ylabel('Población infectada')
plt.title('Comparación de modelos SIR, ABM y Híbrido')
plt.legend()
plt.grid()
plt.show()
